# Setup

In [ ]:
# %pip install -r requirements.txt

# Environment Variables

In [ ]:
# Get environment variables

from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Utilities

In [ ]:
# Import the utils module

from utils import chunk_docs_semantic

ModuleNotFoundError: No module named 'david'

In [8]:
import sys
print(sys.path)

['/opt/anaconda3/envs/era4/lib/python312.zip', '/opt/anaconda3/envs/era4/lib/python3.12', '/opt/anaconda3/envs/era4/lib/python3.12/lib-dynload', '', '/opt/anaconda3/envs/era4/lib/python3.12/site-packages']


# Pre-Processing

In [ ]:
# Get the docs
docs = process_directory(path="docs/10k/html", glob="**/*.html")
print(f"\nNumber of docs = {len(docs)}")

# Chunk the docs
chunks = chunk_docs_recursive(docs=docs)
print(f"\nNumber of chunks = {len(chunks)}")

In [ ]:
# Create embeddings
embeddings = create_embeddings_openai()

# Create a vector store
vector_store = create_vector_store_qdrant(location=':memory:', collection_name='test-1a', vector_size=1536, embeddings=embeddings, docs=chunks)
print(f"\nCreated a vector store")

# Create a retriever
retriever = create_retriever_qdrant(vector_store)
print(f"\nCreated a retriever")

# Create the prompt templage
chat_prompt_template = create_chat_prompt_template()
print(f"\nCreated a prompt template")

# Test 1a - OpenAI and Qdrant with Recursive Chunking

In [ ]:
# Create the chain using naive retrieval
def run_test_1a():
	chain = create_chain_openai(model='gpt-4o', prompt_template=chat_prompt_template, retriever=retriever)
	print(f"\nCreated a chain")

# Test 1b - OpenAI and QDrant using Cohere

In [ ]:
# Create the chain using Cohere contextual compression
chain = create_chain_openai(model='gpt-4o', prompt_template=chat_prompt_template, retriever=retriever, use_cohere=True)
print(f"\nCreated a chain")

# Post-Processing

In [ ]:
# Test the chain with a few questions 
questions = [
	"What is the annual revenue of Uber?",
	"What is the annual revenue of Lyft?",
	"How does Uber's revenue compare to Lyft's revenue?",
	"Summarize Lyft's risk factors",
	]

for question in questions:
	print(question)
	result = chain.invoke({"question" : question})
	print(result)
	print(result["response"].content)
	print("\n*****")

In [ ]:
# Evaluate the chain using Ragas
ragas_results, ragas_results_df = run_ragas_evaluation(chain, testset_name="testsets/10k_testset.csv", eval_metrics=[answer_correctness, answer_relevancy, context_precision, context_recall, faithfulness,])
    
# Write the results to disk
ragas_results_df.to_csv(f"evaluations/10x_test1_testset_evaluation_{get_time_string()}.csv")

# Show the summary resutls
print(ragas_results)

In [ ]:
# Build a heatmap showing the Ragas results

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap

heatmap_data = ragas_results_df[
	[
		"answer_correctness",
		"answer_relevancy",
		"context_precision", 
		"context_recall", 
		"faithfulness",]
	]

cmap = LinearSegmentedColormap.from_list("green_red", ["red", "green"])

plt.figure(figsize=(10, 8))
sns.heatmap(heatmap_data, annot=True, fmt=".2f", linewidths=0.5, cmap=cmap)
plt.yticks(ticks=range(len(ragas_results_df["user_input"])), labels=ragas_results_df["user_input"].str[:100], rotation=0)
plt.show()